<a href="https://colab.research.google.com/github/Nohgyumin0812/China-Korea-correlation-trading-/blob/main/%5B%EC%9D%BC%ED%9A%8C%EC%9A%A9%5D_%EC%A4%91%EA%B5%AD_%EA%B5%AD%EB%82%B4%EC%A2%85%EB%AA%A9_%EA%B0%84_%EC%83%81%EA%B4%80%EA%B4%80%EA%B3%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install finance-datareader
!pip install --upgrade finance-datareader

import yfinance
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import FinanceDataReader as fdr

from matplotlib import font_manager, rc

# 한글 폰트 설정
font_path = 'C:/Users/wmals/OneDrive/바탕 화면/글꼴/NanumGothic.ttf'  # 나눔고딕 폰트 경로
font = font_manager.FontProperties(fname=font_path).get_name()
plt.rcParams['axes.unicode_minus'] = False
rc('font', family=font)


stock = fdr.StockListing('KRX')
#stock = fdr.StockListing('KRX')
Marcap_condition = 500000000000
stock = stock[stock['Marcap'] > Marcap_condition]
start = '2022-01-01'
end = datetime.datetime.now()
stock['Code'] = stock.apply(lambda row : row['Code'] + '.KS' if row['Market'] == 'KOSPI' else row['Code'] + '.KQ', axis = 1)

stock_list = stock['Code'].to_list()
name_list = stock['Name'].to_list()
price = yfinance.download(stock_list, start, end)['Close']

hsi = '^HSI'

# yfinance로 항셍지수 데이터 다운로드
hsi_data = yfinance.download(hsi, start, end)['Close']

hsi_data.index = pd.to_datetime(hsi_data.index).tz_localize(None)
price.index = pd.to_datetime(price.index).tz_localize(None)
merge_price = pd.concat([price, hsi_data], axis = 1)
corr_list = []


for ticker in stock_list:
  corr = price[ticker].corr(hsi_data)
  corr_list.append(corr)

corr_df = pd.DataFrame(stock_list, index = name_list, columns = ['Code'])
corr_df['2years_correlation'] = corr_list
corr_df = corr_df.sort_values(by = '2years_correlation', ascending = False).head(20)

ret_year2_list = []
for ticker in corr_df['Code'].to_list():
  ret_year2 = price[ticker].pct_change(504).iloc[-1]
  ret_year2_list.append(ret_year2)
corr_df['2years_return'] = ret_year2_list
name_list = corr_df.index.to_list()

disparate_ratio_list = []
fig, axes = plt.subplots(30, 4, figsize=(40, 180))
axes = axes.flatten()
for i, ticker in enumerate(corr_df['Code'].to_list()):
    ax = axes[i]
    symbol_pair = [ticker, hsi]
    spread = np.log(merge_price[symbol_pair[0]]) - np.log(merge_price['Close']).dropna()
    print(spread)
    spread_mean = spread.mean()
    leader_name = name_list[i]
    follower_name = symbol_pair[1]
    upper_spread = (spread.rolling(window=20).mean() + 2 * spread.rolling(window=20).std()).dropna()
    lower_spread = (spread.rolling(window=20).mean() - 2 * spread.rolling(window=20).std()).dropna()
    spread_1std_upper = spread.mean() + spread.std()
    spread_1std_lower = spread.mean() - spread.std()
    spread_2std_upper = spread.mean() + 2 * spread.std()
    spread_2std_lower = spread.mean() - 2 * spread.std()
    spread_3std_upper = spread.mean() + 3 * spread.std()
    spread_3std_lower = spread.mean() - 3 * spread.std()
    ax.plot(spread.interpolate(method='linear'), label='Log_Spread', color='Navy')
    ax.axhline(y=spread_mean, color='red', linestyle='-', label=' Log_Spread_Mean')
    text_symbol_1 = f'↑{leader_name}강세'
    text_symbol_2 = f"↓{follower_name}강세"
    # ax.text(half_date, spread.max(), text_symbol_1, fontsize = 12, color = 'green', ha = 'center')
    # ax.text(half_date, spread.min(), text_symbol_2, fontsize = 12, color = 'red', ha = 'center')
    ax.fill_between(spread.index, spread_1std_lower, spread_1std_upper, color='green', alpha=0.8, label='1σ Range')
    ax.fill_between(spread.index, spread_2std_lower, spread_2std_upper, color='gray', alpha=0.5, label='2σ Range')
    ax.fill_between(spread.index, spread_3std_lower, spread_3std_upper, color='skyblue', alpha=0.5, label='3σ Range')

    #ax.fill_between(spread.index, upper_spread, lower_spread, color='red', alpha=0.3, label='micro_2σ Range')

    ax.set_title(f'Daily Log Spread\n ↑{leader_name}강세 / ↓{follower_name}강세')
   # ax.set_xlabel('Date')
    #ax.set_ylabel('Log Spread')
    ax.legend()
    ax.grid(True)

    disparate_ratio = spread.iloc[-1] / spread_mean
    disparate_ratio_list.append(disparate_ratio)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.savefig('섹터별 페어_시각화.png', dpi=300)
plt.show()

corr_df['2years_disparate_ratio'] = disparate_ratio_list
corr_df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 3.7 MB/s eta 0:00:00


FileNotFoundError: [Errno 2] No such file or directory: '/content/C:/Users/wmals/OneDrive/바탕 화면/글꼴/NanumGothic.ttf'